Here is analysis of the performance of different filters

In [ ]:
##Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
def low_pass_filter(data, alpha):
    """
    Apply a simple digital low-pass filter to a 1D array or pandas Series.
    
    Parameters:
    data (array-like): Input data to be filtered.
    alpha (float): Smoothing factor, between 0 and 1.
    
    Returns:
    filtered_data (array-like): Filtered data.
    """
    filtered_data =[data[0]]
     # Initialize with the first data point
    for t in range(1, len(data)):
        filtered_value = alpha * data[t] + (1 - alpha) * filtered_data[t-1]
        filtered_data.append(filtered_value)
        print(t)
    filtered_data = pd.Series(filtered_data, index=data.index)

    return filtered_data

In [ ]:
def adaptive_low_pass_filter(data, color_count):
    """
    Apply a simple digital low-pass filter to a 1D array or pandas Series.
    
    Parameters:
    data (array-like): Input data to be filtered.
    alpha (float): Smoothing factor, between 0 and 1.
    
    Returns:
    filtered_data (array-like): Filtered data.
    """
   
    filtered_data =[data[0]]
     # Initialize with the first data point
    for t in range(1, len(data)):
        alpha=0.065/(12/np.sqrt(color_count[t]+1))
        filtered_value = alpha * data[t] + (1 - alpha) * filtered_data[t-1]
        filtered_data.append(filtered_value)
        print(t)
    filtered_data = pd.Series(filtered_data, index=data.index)

    return filtered_data

In [ ]:
folder_path = '/home/myedut/Downloads/sinewave_old/'

# Initialize the list
chenyao_data_raw_3 = []

# Use glob to find all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read each CSV file and append to the list
for file in csv_files:
    chenyao_data_raw_3.append(pd.read_csv(file))

folder_path = '/home/myedut/Downloads/sinewave_new/'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))



for file in csv_files:
    chenyao_data_raw_3.append(pd.read_csv(file))

In [ ]:
###Pick ranges that contain flying with sinewave-modulated acceleration mu_x
ranges=[(324,710),(100,420),(100,620),(100,710),(100,600)]

ranges_2=[(50,250),(50,400),(50,250),(50,300),(60,360),(60,360),(60,310),(60,310)]

ranges_3=ranges+ranges_2

ranges_2=ranges_3

In [ ]:
#Downsample the data
chenyao_data_resampled = []

# Ensure there is no variable named 'list' that shadows the built-in list type
# If you have such a variable, rename it to something else

for i in range(len(chenyao_data_raw_3)):
    print(i)
    chenyao_data_raw_3[i]['time'] = pd.to_timedelta(chenyao_data_raw_3[i]['time'], unit='s')
    chenyao_data_raw_3[i].set_index('time', inplace=True)

    chenyao_data_resampled.append(chenyao_data_raw_3[i].resample('65ms').mean())

In [ ]:
box_location_1=np.array([4,-.5,-0.7])

box_location_2=np.array([4.4,2.1,-1])
###Calculate the absolute distance to the landing target
for i in range(len(chenyao_data_resampled)):
    if i <5:
        box_location=box_location_1   
        true_x=(chenyao_data_resampled[i]['pos_x'])-box_location[0]
        true_y=chenyao_data_resampled[i]['pos_y']-box_location[1]
        true_z=chenyao_data_resampled[i]['pos_z']-box_location[2]
        chenyao_data_resampled[i]['abs_distance']=np.sqrt(true_x**2+true_y**2+true_z**2)
    else:
        box_location=box_location_2
        true_x=(chenyao_data_resampled[i]['pos_x'])-box_location[0]
        true_y=chenyao_data_resampled[i]['pos_y']-box_location[1]
        true_z=chenyao_data_resampled[i]['pos_z']-box_location[2]
        chenyao_data_resampled[i]['abs_distance']=np.sqrt(true_x**2+true_y**2+true_z**2)

#Calculate ground truth divergence based on your distance to the target and velocity. We first calculate velocity towards the box and then convert it to divertgence
for i in range(len(chenyao_data_resampled)):
    if i <5:
        box_location=box_location_1
        approach=((chenyao_data_resampled[i]['pos_x']-box_location[0])*chenyao_data_resampled[i]['vel_x'] + (chenyao_data_resampled[i]['pos_y']-box_location[1])*chenyao_data_resampled[i]['vel_y']+(chenyao_data_resampled[i]['pos_z']-box_location[2])*chenyao_data_resampled[i]['vel_z'])/chenyao_data_resampled[i]['abs_distance']
        chenyao_data_resampled[i]['real_divergence']=-1*approach/chenyao_data_resampled[i]['abs_distance']
    else:
        box_location=box_location_2
        approach=((chenyao_data_resampled[i]['pos_x']-box_location[0])*chenyao_data_resampled[i]['vel_x'] + (chenyao_data_resampled[i]['pos_y']-box_location[1])*chenyao_data_resampled[i]['vel_y']+(chenyao_data_resampled[i]['pos_z']-box_location[2])*chenyao_data_resampled[i]['vel_z'])/chenyao_data_resampled[i]['abs_distance']
        chenyao_data_resampled[i]['real_divergence']=-1*approach/chenyao_data_resampled[i]['abs_distance']
  

In [ ]:
### code to calculate alpha for the lowpass filter. We need it, because in each flight alpha can be different
dt=0.065
filter_constant=12

alpha_20=[]

for i in range(len(chenyao_data_resampled)):
    count=np.sqrt(chenyao_data_resampled[i]['color_count'].iloc[ranges_2[i][0]:ranges_2[i][1]].quantile(0.2))
    time_constant=filter_constant/count
    
    print(time_constant)
    alpha_20.append(dt/(time_constant))
alpha_20=np.array(alpha_20)

In [ ]:
###code to calculate alpha for the lowpass filter. We need it, because in each flight alpha can be different
dt=0.065
filter_constant=12

alpha_80=[]

for i in range(len(chenyao_data_resampled)):
    count=np.sqrt(chenyao_data_resampled[i]['color_count'].iloc[ranges_2[i][0]:ranges_2[i][1]].quantile(0.8))
    time_constant=filter_constant/count
    
    print(time_constant)
    alpha_80.append(dt/(time_constant))
alpha_80=np.array(alpha_80)

In [ ]:
for i in range(len(chenyao_data_resampled)):
    filtered_data_20=[chenyao_data_resampled[i]['raw_divergence'].iloc[0]]
    filtered_data_80=[chenyao_data_resampled[i]['raw_divergence'].iloc[0]]
     # Initialize with the first data point
    #filtered_data = [chenyao_data_resampled[0]['raw divergence']]  # Initialize with the first data point
    
    for t in range(1, len(chenyao_data_resampled[i])):
        filtered_value_20 = alpha_20[i] * chenyao_data_resampled[i]['raw_divergence'].iloc[t] + (1 - alpha_20[i]) * filtered_data_20[t-1]
        
        filtered_data_20.append(filtered_value_20)

        filtered_value_80 = alpha_80[i] * chenyao_data_resampled[i]['raw_divergence'].iloc[t] + (1 - alpha_80[i]) * filtered_data_80[t-1]
        filtered_data_80.append(filtered_value_80)
    chenyao_data_resampled[i]['divergence_constant_20']=np.array(filtered_data_20)
    chenyao_data_resampled[i]['divergence_constant_80']=np.array(filtered_data_80)

In [ ]:
errors_constant_20=[]
errors_constant_80=[]
errors_adaptive=[]
for i in range(len(chenyao_data_resampled)):
    errors_constant_20.append(np.mean(np.abs(chenyao_data_resampled[i]['divergence_constant_20'].values[ranges_2[i][0]+4:ranges_2[i][1]+4]-chenyao_data_resampled[i]['real_divergence'].values[ranges_2[i][0]:ranges_2[i][1]])))
    errors_adaptive.append(np.mean(np.abs(chenyao_data_resampled[i]['divergence'].values[ranges_2[i][0]+4:ranges_2[i][1]+4]-chenyao_data_resampled[i]['real_divergence'].values[ranges_2[i][0]:ranges_2[i][1]])))
    errors_constant_80.append(np.mean(np.abs(chenyao_data_resampled[i]['divergence_constant_80'].values[ranges_2[i][0]+4:ranges_2[i][1]+4]-chenyao_data_resampled[i]['real_divergence'].values[ranges_2[i][0]:ranges_2[i][1]])))

In [ ]:
errors_constant_20=np.array(errors_constant_20)
errors_adaptive=np.array(errors_adaptive)
errors_constant_80=np.array(errors_constant_80)

errors_adaptive/errors_constant_20

In [ ]:
###Statistical comparison
import seaborn as sns 
plt.figure(figsize=(8,8)) 
 
data_box_plot={'Adaptive Filter':errors_adaptive,'Constant 80':errors_constant_80,'Constant 20':errors_constant_20}  
sns.boxplot(data=pd.DataFrame(data_box_plot), palette=['green','dodgerblue','purple']) 
# Plot boxplots for WT and mutant separately 
#sns.boxplot(x='WT',data=df_wt, color='grey') 
#sns.boxplot(x='Mutant',data=df_mutant, color='red') 
 
# Add individual data points with jitter for better visibility 
#sns.stripplot(data=df_wt, color='black', jitter=True, alpha=0.5) 
#sns.stripplot(data=df_mutant, color='black', jitter=True, alpha=0.5) 
 
# Customize fonts and labels 
sns.stripplot(data=pd.DataFrame(data_box_plot).abs(), color='black', jitter=True, alpha=1) 
plt.ylim([0.00, 0.05]) 
# Customize fonts and labels 
plt.xlabel('') 
plt.ylabel('Divergence Absolute Error', fontsize=22) 
plt.title('D. Statistical Filter Comparison', fontsize=26,y=1.07) 
 
# Adjust font sizes 
plt.xticks(fontsize=22) 
plt.yticks(fontsize=22) 
 
 
# Remove spines on the top and right 
plt.gca().spines['top'].set_visible(False) 
plt.gca().spines['right'].set_visible(False) 
 
plt.xticks([0, 1.1,2.2], ['Adaptive', ' Static-80','Static-20'],fontsize=22) 
 
# Adjust the position of x-tick labels
for label in plt.gca().get_xticklabels():
    label.set_y(-0.05)  # Adjust this value to move the labels lower

#plt.legend(['WT','','Ano2-/-'], fontsize=12) 
# Add legend with custom marker color 
#legend = plt.legend(['WT', 'Mutant'], fontsize=12) 
#legend.legendHandles[1].set_color('red')   
#plt.savefig('/home/myedut/Documents/Servoing_paper/Summary Statistics_2.png', dpi=300, bbox_inches='tight') 
# Show plot 
plt.show()

In [ ]:
from scipy.stats import ttest_1samp

adaptive_values = united_set['Adaptive Filter']
constant_80_values = united_set['Constant 80']
constant_20_values = united_set['Constant 20']

# Calculate the differences
diff_80 = adaptive_values - constant_80_values
diff_20 = adaptive_values - constant_20_values

# Perform one-sample t-test
t_stat_80, p_value_80 = ttest_1samp(diff_80, 0)
t_stat_20, p_value_20 = ttest_1samp(diff_20, 0)

print(f"One-sample t-test between 'Adaptive' and '80': t-statistic = {t_stat_80}, p-value = {p_value_80}")
print(f"One-sample t-test between 'Adaptive' and '20': t-statistic = {t_stat_20}, p-value = {p_value_20}")

In [ ]:
import matplotlib.ticker as ticker
i = 8
plt.figure(figsize=(12, 8))

# Assuming 'abs_distance' is the column representing the distance to the target
distance_to_target = chenyao_data_resampled[i]['abs_distance'].iloc[ranges_2[i][0]:ranges_2[i][1]].reset_index(drop=True)

# Adjust the index to start from zero
raw_divergence = chenyao_data_resampled[i]['raw_divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)
real_divergence = chenyao_data_resampled[i]['real_divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].reset_index(drop=True)
divergence_constant_80 = chenyao_data_resampled[i]['divergence_constant_80'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)
divergence_constant_20 = chenyao_data_resampled[i]['divergence_constant_20'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-6).reset_index(drop=True)
divergence = chenyao_data_resampled[i]['divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)


# Plot the data versus distance to the target
plt.plot(distance_to_target, raw_divergence, color='grey', linewidth=1, label='raw divergence')
plt.plot(distance_to_target, real_divergence, color='coral', linewidth=3, label='ground truth divergence')
plt.plot(distance_to_target, divergence_constant_80, color='dodgerblue', linewidth=3, label='constant filter (80)')
plt.plot(distance_to_target, divergence_constant_20, color='purple', linewidth=3, label='constant filter (20)')
plt.plot(distance_to_target, divergence, color='green', linewidth=3, label='adaptive filter')

plt.legend(fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Distance to target (m)', fontsize=22)
plt.ylabel('Divergence', fontsize=22)
ax = plt.gca()
plt.gca().invert_xaxis() # Invert x-axis from 5 to 1

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_locator(ticker.MaxNLocator(6))  # Fewer x-ticks
plt.title('A. Comparison of different filtering approaches', fontsize=26)
#plt.savefig('/home/myedut/Documents/Servoing_paper/Visuafiltering_comparison_Adaptive Filter: All Filters.png', bbox_inches='tight',dpi=300)
#plt.xlim([400,700])
plt.show()

In [ ]:
import matplotlib.ticker as ticker
i = 8
plt.figure(figsize=(12, 8))

# Assuming 'abs_distance' is the column representing the distance to the target
distance_to_target = chenyao_data_resampled[i]['abs_distance'].iloc[ranges_2[i][0]:ranges_2[i][1]].reset_index(drop=True)

# Adjust the index to start from zero
raw_divergence = chenyao_data_resampled[i]['raw_divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)
real_divergence = chenyao_data_resampled[i]['real_divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].reset_index(drop=True)
#divergence_constant_80 = chenyao_data_resampled[i]['divergence_constant_80'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)
#divergence_constant_20 = chenyao_data_resampled[i]['divergence_constant_20'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-6).reset_index(drop=True)
divergence = chenyao_data_resampled[i]['divergence'].iloc[ranges_2[i][0]:ranges_2[i][1]].shift(-4).reset_index(drop=True)


# Plot the data versus distance to the target
plt.plot(distance_to_target, raw_divergence, color='grey', linewidth=1, label='raw divergence')
plt.plot(distance_to_target, real_divergence, color='coral', linewidth=3, label='ground truth divergence')
#plt.plot(distance_to_target, divergence_constant_80, color='dodgerblue', linewidth=3, label='constant filter (80)')
#plt.plot(distance_to_target, divergence_constant_20, color='purple', linewidth=3, label='constant filter (20)')
plt.plot(distance_to_target, divergence, color='green', linewidth=3, label='adaptive filter')

plt.legend(fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Distance to target (m)', fontsize=22)
plt.ylabel('Divergence', fontsize=22)
ax = plt.gca()
plt.gca().invert_xaxis() # Invert x-axis from 5 to 1

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_locator(ticker.MaxNLocator(6))  # Fewer x-ticks
plt.title('B. Performance of Adaptive Filter', fontsize=26)
#plt.savefig('/home/myedut/Documents/Servoing_paper/Visuafiltering_comparison_Adaptive Filter:Adaptive Filter.png', bbox_inches='tight',dpi=300)
#plt.xlim([400,700])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
i = 8
plt.figure(figsize=(12, 8))

# Assuming 'abs_distance' is the column representing the distance to the target
distance_to_target = chenyao_data_resampled[i]['abs_distance'].iloc[ranges_2[i][0]:ranges_2[i][1]].reset_index(drop=True)
# Extract the color count data in the specified range
color_count_data = chenyao_data_resampled[i]['color_count'].iloc[ranges_2[i][0]:ranges_2[i][1]]

# Calculate the time constant based on color count
time_constant = 12 / np.sqrt(color_count_data)

# Calculate the mean and median of the time constant
time_constant_80 = 0.065/(alpha_80[i])
time_constant_20 = 0.065/(alpha_20[i])

# Plotting only the time constants
plt.figure(figsize=(12, 8))
plt.plot(distance_to_target,time_constant,color='green', linewidth=3, label=r'$\tau$-adaptive')

# Plot the mean and median lines for the time constant
plt.axhline(time_constant_20, color='purple', linestyle='--', linewidth=3, label=r'$\tau$-20')
plt.axhline(time_constant_80, color='dodgerblue', linestyle='--', linewidth=3, label=r'$\tau$-80')

# Adding labels, legend, and plot settings
plt.legend(fontsize=24)
plt.title('C. Adaptive and Fixed Time Constants', fontsize=26)
plt.xlabel('Distance to target (m)', fontsize=22)
plt.ylabel('Time Constant', fontsize=22)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.grid(False)

# Customize plot spines and ticks
ax = plt.gca()
plt.gca().invert_xaxis() # Invert x-axis from 5 to 1

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_locator(ticker.MaxNLocator(6))

# Save plot as image
#plt.savefig('/home/myedut/Documents/Servoing_paper/time_constant_only.png', dpi=300)
plt.show()